### Mini batch
- 기존에 사용했던 전체 데이터를 대상으로 한 번에 경사 하강법을 수행하는 방법은 '배치 경사 하강법'이라 한다.
- 배치 경사 하강법은 전체 데이터를 사용하므로 W가 최적값에 수렴하는 과정이 안정적이다.
- 하지만 시간이 너무 오래 걸리기 때문에, 나누어서 하는 방법이 필요하고 이를 '미니 배치 경사 하강법'이라 한다.
- 미니 배치 경사 하강법은 미니 배치 단위로 경사 하강법을 수행하는 방법이다.
- 전체가 아닌 일부 데이터를 사용하기 때문에 W가 최적값에 수렴하기 위해 많이 헤맬 수 있다.
- 하지만 상대적으로 훈련 속도가 빨라서 시간적 효율이 높다.

<img src="./images/mini_batch.png" width="500px">

In [28]:
import pandas as pd

pre_m_df = pd.read_csv('./datasets/medical_cost.csv')
pre_m_df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [29]:
from sklearn.preprocessing import LabelEncoder

pre_m_df = m_df.copy()
columns = ['sex', 'smoker']
label_encoders = {}

for column in columns:
    encoder = LabelEncoder()
    result = encoder.fit_transform(pre_m_df[column])
    label_encoders[column] = encoder.classes_
    pre_m_df[column] = result

label_encoders

{'sex': array(['female', 'male'], dtype=object),
 'smoker': array(['no', 'yes'], dtype=object)}

In [30]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# sparse_output를 False로 설명하면 넘파이 배열, True는 희소 행렬
one_hot_encoder = OneHotEncoder(sparse_output=False)
result = one_hot_encoder.fit_transform(pre_m_df[['region']])



pre_m_df = pd.concat([pre_m_df, pd.DataFrame(result, columns= one_hot_encoder.categories_).astype(np.int8)], axis=1)
pre_m_df

,age,sex,bmi,children,smoker,region,charges,"(northeast,)","(northwest,)","(southeast,)","(southwest,)"
0,19,0,27.900,0,1,southwest,16884.92400,0,0,0,1
1,18,1,33.770,1,0,southeast,1725.55230,0,0,1,0
2,28,1,33.000,3,0,southeast,4449.46200,0,0,1,0
3,33,1,22.705,0,0,northwest,21984.47061,0,1,0,0
4,32,1,28.880,0,0,northwest,3866.85520,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1333,50,1,30.970,3,0,northwest,10600.54830,0,1,0,0
1334,18,0,31.920,0,0,northeast,2205.98080,1,0,0,0
1335,18,0,36.850,0,0,southeast,1629.83350,0,0,1,0
1336,21,0,25.800,0,0,southwest,2007.94500,0,0,0,1


In [31]:
pre_m_df = pre_m_df.drop(labels=['region'], axis=1)

pre_m_df.columns = ['age', 'sex', 'bmi', 'children', 'smoker', 'charges', 'northeast', 'northwest', 'southeast', 'southwest']

In [32]:
pre_m_df

,age,sex,bmi,children,smoker,charges,northeast,northwest,southeast,southwest
0,19,0,27.900,0,1,16884.92400,0,0,0,1
1,18,1,33.770,1,0,1725.55230,0,0,1,0
2,28,1,33.000,3,0,4449.46200,0,0,1,0
3,33,1,22.705,0,0,21984.47061,0,1,0,0
4,32,1,28.880,0,0,3866.85520,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
1333,50,1,30.970,3,0,10600.54830,0,1,0,0
1334,18,0,31.920,0,0,2205.98080,1,0,0,0
1335,18,0,36.850,0,0,1629.83350,0,0,1,0
1336,21,0,25.800,0,0,2007.94500,0,0,0,1


In [20]:
np.log1p(pre_m_df['charges']).hist()

0        9.734236
1        7.453882
2        8.400763
3        9.998137
4        8.260455
          ...    
1333     9.268755
1334     7.699381
1335     7.396847
1336     7.605365
1337    10.279948
Name: charges, Length: 1338, dtype: float64

In [33]:
pre_m_df.loc[:,'charges'] = np.log1p(pre_m_df['charges'])


In [34]:
pre_m_df.corr()['charges'].sort_values(ascending=False)[1:]

smoker       0.665539
age          0.527807
children     0.161317
bmi          0.132678
northeast    0.043109
southeast    0.015803
sex          0.005644
northwest   -0.017830
southwest   -0.041633
Name: charges, dtype: float64

In [36]:
columns = ['smoker', 'age', 'children', 'bmi', 'charges']
pre_m_df = pre_m_df.loc[:, columns]
pre_m_df

,smoker,age,children,bmi,charges
0,1,19,0,27.900,9.734236
1,0,18,1,33.770,7.453882
2,0,28,3,33.000,8.400763
3,0,33,0,22.705,9.998137
4,0,32,0,28.880,8.260455
...,...,...,...,...,...
1333,0,50,3,30.970,9.268755
1334,0,18,0,31.920,7.699381
1335,0,18,0,36.850,7.396847
1336,0,21,0,25.800,7.605365


In [42]:
from sklearn.model_selection import train_test_split
import numpy as np
import torch
from torch.nn import Linear
from torch.nn.functional import mse_loss
from torch.optim import SGD
from torch.utils.data import TensorDataset, DataLoader

torch.manual_seed(124)

features, targets = pre_m_df.iloc[:, :-1], pre_m_df.iloc[:, -1]

X_train, X_test, y_train, y_test = \
train_test_split(features, targets, test_size=0.2, random_state=124)

X_train = torch.FloatTensor(X_train.values)
y_train = torch.FloatTensor(y_train.values).view(-1, 1)

X_test = torch.FloatTensor(X_test.values)
y_test = torch.FloatTensor(y_test.values).view(-1, 1)

td = TensorDataset(X_train, y_train)
dl = DataLoader(td, batch_size=100, shuffle=True)

l_r = Linear(4,1)

optimizer = SGD(l_r.parameters(), lr= 0.01)

epochs = 1000

for epoch in range(1, epochs+1):
    for i, samples in enumerate(dl):
        X_train, y_train = samples

        H = l_r(X_train)
        loss = mse_loss(H, y_train)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % 100 == 0:
            print(f'Epoch: {epoch}/{epochs}, Batch: {i +1}/{len(dl)}')






Epoch: 100/1000, Batch: 0/2
Epoch: 100/1000, Batch: 1/2
Epoch: 100/1000, Batch: 2/2
Epoch: 100/1000, Batch: 3/2
Epoch: 100/1000, Batch: 4/2
Epoch: 100/1000, Batch: 5/2
Epoch: 100/1000, Batch: 6/2
Epoch: 100/1000, Batch: 7/2
Epoch: 100/1000, Batch: 8/2
Epoch: 100/1000, Batch: 9/2
Epoch: 100/1000, Batch: 10/2
Epoch: 200/1000, Batch: 0/2
Epoch: 200/1000, Batch: 1/2
Epoch: 200/1000, Batch: 2/2
Epoch: 200/1000, Batch: 3/2
Epoch: 200/1000, Batch: 4/2
Epoch: 200/1000, Batch: 5/2
Epoch: 200/1000, Batch: 6/2
Epoch: 200/1000, Batch: 7/2
Epoch: 200/1000, Batch: 8/2
Epoch: 200/1000, Batch: 9/2
Epoch: 200/1000, Batch: 10/2
Epoch: 300/1000, Batch: 0/2
Epoch: 300/1000, Batch: 1/2
Epoch: 300/1000, Batch: 2/2
Epoch: 300/1000, Batch: 3/2
Epoch: 300/1000, Batch: 4/2
Epoch: 300/1000, Batch: 5/2
Epoch: 300/1000, Batch: 6/2
Epoch: 300/1000, Batch: 7/2
Epoch: 300/1000, Batch: 8/2
Epoch: 300/1000, Batch: 9/2
Epoch: 300/1000, Batch: 10/2
Epoch: 400/1000, Batch: 0/2
Epoch: 400/1000, Batch: 1/2
Epoch: 400/1000, 